# imports

In [1]:
import foldermerge
from pathlib import Path

root = Path().absolute().parent / "tests"
foldermerge.core.RESULTS_PATH = root / "results"
foldermerge.clear_results()

In [1]:
%load_ext line_profiler

In [1]:
import foldermerge
from pathlib import Path

root = Path(r"C:\Users\timot\Documents\NasgoyaveOC")
copy = Path(r"F:\NasgoyaveOC_from_Pasteur\Administratif")
fm = foldermerge.FolderMerger(root, copy, refresh = False)

loading <FolderChecker ce3424bc at C:\Users\timot\Documents\NasgoyaveOC>
loading <FolderChecker ac8e865e at F:\NasgoyaveOC_from_Pasteur\Administratif>
saving <FolderChecker ce3424bc at C:\Users\timot\Documents\NasgoyaveOC>
saving <FolderChecker ac8e865e at F:\NasgoyaveOC_from_Pasteur\Administratif>
loading <FolderComparator ac8e865e_vs_ce3424bc>
saving <FolderComparator ac8e865e_vs_ce3424bc>


## Continue here

In [15]:
from flask import Flask, render_template_string, request
import pandas as pd, numpy as np
import json

app = Flask(__name__)

@app.route('/')
def index():

    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Material Style Foldable Folder Structure</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <style>
    </style>
    <script>
    function dropHandler(ev) {
        ev.preventDefault();
        
        if (ev.dataTransfer.items && ev.dataTransfer.items[0].webkitGetAsEntry().isDirectory) {
            var folder = ev.dataTransfer.items[0].getAsFile();
            $('#reference_folder').val(folder.name);
        }
    }

    function dragOverHandler(ev) {
        ev.preventDefault();
    }

    function folderChangeHandler(ev) {
        var files = ev.target.files;
        if (files.length) {
            // Assuming single folder selection, take the first file's path
            // Remove the file name (last part of the path) to get the folder path
            var folderPath = files[0].webkitRelativePath.split('/').slice(0, -1).join('/');
            $('#reference_folder').val(folderPath);
        }
    }
    </script>   
    <body>
        <span>BONJOUR</span>
        <form method="post" action="/view_results">
            <input type="text" name="reference_folder" id="reference_folder" readonly>

            <div id="drop_zone" ondrop="dropHandler(event);" ondragover="dragOverHandler(event);">
                Drag and drop a folder here or click <b>Browse</b> to select a folder.
            </div>

            <input type="file" id="folderPicker" name="folder" webkitdirectory directory style="display: none;" onchange="folderChangeHandler(event);">
            <!-- Browse Button -->
            <button type="button" onclick="document.getElementById('folderPicker').click();">Browse</button>
  
            <input type="text" name="compared_folders" />
            <input type="checkbox" name="refresh" value="true" id="refresh" />
            <label for="refresh">Refresh</label>
            <input type="submit" value="Submit" />
        </form>
    </body>
    </html>
    """

    return render_template_string(html_template)

@app.route('/view_results', methods=['POST'])
def view_results():
    reference_folder = Path(request.form['reference_folder'])
    compared_folders = request.form.get('compared_folders', "")
    refresh = json.loads(request.form.get('refresh', "false"))

    print(reference_folder)
    print(compared_folders)
    print(refresh)

    fm = foldermerge.FolderMerger(reference_folder, compared_folders, refresh = refresh)
    report = fm.report()
    print(report)

@app.route('/view_inexistant_files', methods=['POST'])
def view_inexistant_files():
    reference_folder = request.form['reference_folder']
    compared_folders = request.form['compared_folders']
    refresh = json.loads(request.form['refresh'])

    fm = foldermerge.FolderMerger(reference_folder, compared_folders, refresh = refresh)
    
    df = fm.folders.child(0).comparisons[fm.folders.main.name].get_inexistant_files()

    # Convert DataFrame to a nested dictionary that represents the folder structure
    tree = {}
    for _, row in df.iterrows():
        parts = row['reldirpath'].split('\\')
        current_level = tree
        for part in parts:
            if part not in current_level:
                current_level[part] = {}
            current_level = current_level[part]
        current_level[row['name']] = {'fullpath': row['fullpath'], 'hash': row['hash']}
    
    # Define a recursive function for the nested structure template
    def render_tree(tree):
        html = ""#'<ul class="folder-structure">'
        for folder, contents in tree.items():
            if isinstance(contents, dict) :
                if "fullpath" in contents.keys() : 
                    # Handle the case where contents is dictionary representing a file
                    html += '<table class="file-content">'
                    for key, value in contents.items():
                        html += f'<tr><td class="category_key">{key}</td><td> : </td><td class="category_value">{value}</td></tr>'
                    html += '</table>'
                else :
                    # contents is a dictionary (subfolder)
                    html += f'<li class="folder">{folder}</li>'
                    html += '<ul class="folder-content">'
                    html += render_tree(contents)  # Recursively render subdirectories
                    html += '</ul>'
            else : 
                continue                
        #html += '</ul>'
        return html

    print(tree)

    # Render the tree using the recursive function
    tree_html = render_tree(tree)


    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Material Style Foldable Folder Structure</title>
    <style>
    body {
        font-family: 'Roboto', sans-serif;
        background: #f5f5f5;
        margin: 0;
        padding: 0;
    }

    .structure-viewer {
        max-width: 100vw;
        max-height: 100vh;
        margin: 20px;
    }

    .folder-structure {
        margin: 5px;
    }

    .folder-structure .file-content {
        background: #fafafa;
        margin-bottom: 8px;
        border-radius: 4px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        padding: 10px 20px;
    }

    .folder-structure .category_key {
        font-weight: bold;
    }

    folder-structure .category_value {
        font-weight: normal;
    }

    .folder-structure, .folder-structure .folder-content {
        list-style: none;
        padding: 0;
        margin: 0px;
    }
    .folder-structure .folder {
        background: #fff;
        margin-bottom: 8px;
        border-radius: 4px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        cursor: pointer;
        padding: 10px 20px;
    }
    .folder-structure .folder:before {
        content: '▶';
        display: inline-block;
        margin-right: 10px;
        transition: transform 0.3s ease;
        transform: rotate(0deg);
    }
    .folder-structure .folder-content {
        padding-left: 30px;
        display: none;
    }
    .folder-structure .folder.open:before {
        transform: rotate(90deg);
    }
    .folder-structure .folder.open + .folder-content {
        display: block;
    }

    </style>
    </head>
    <body>
    """ + f"""
    <div class="structure-viewer">
        <ul class="folder-structure">
        {tree_html}
        </ul>
    </div>""" + """
    <script>
    //document.addEventListener('DOMContentLoaded', function() {
        document.querySelectorAll('.folder').forEach(function(folder) {

            folder.classList.toggle('open', true);
            // content = this.nextElementSibling.style.display = 'block'
            /* Change true to false and 'none' to 'block' to be unfolded or folder by default */

            folder.addEventListener('click', function() {

                this.classList.toggle('open');    
            
                /*
                var content = this.nextElementSibling;
                if (content.style.display === 'block') {
                    content.style.display = 'none';
                    this.classList.toggle('open',false);
                } else {
                    content.style.display = 'block';
                    this.classList.toggle('open',true);
                }
                */
            });
        });
    //});
    </script>
    </body>
    </html>
    """

    return render_template_string(html_template)

import webbrowser
from threading import Timer

def open_browser():
      webbrowser.open_new('http://127.0.0.1:5000/')

Timer(1, open_browser).start()
app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off
 INFO     : WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
 INFO     : Press CTRL+C to quit
 INFO     : 127.0.0.1 - - [05/Mar/2024 03:36:18] "GET / HTTP/1.1" 200 -


## Algorithm :

| content match | name match | date case      | case name         | action to take               |
| :-----------: | :--------: | :------------: | :---------------: | :--------------------------: |
| True          | True       | more recent    | identical         | *ignore* (keep ref)            |
| True          | True       | more old       | identical         | *ignore* (keep ref)            |
| True          | False      | more recent    | renamed or moved  | **copy** to ref (if context same)|
| True          | False      | more old       | renamed or moved  | *ignore* (keep ref)            |
| False         | True       | more recent    | updated modified  | **copy** to ref (overwrite)      |
| False         | True       | more old       | outdated modified | *ignore* (keep ref)            |
| False         | False      | -              | -                 | **copy** to ref (no file there)  |

Details for renamed or moved :

| date case   | context details | case name | action to take |
| :---------: | :-------------: | :-------: | :------------: |
| more recent | tbd             |           |                |
| more old    | tbd             |           |                |